In [1]:
import numpy as np
import pandas as pd
from itertools import combinations

In [2]:
#Get and prepair Dataset 
Data = pd.read_csv('groceriesList.csv')
dataraw = len(Data)
datacol = len(Data.columns)


In [3]:
# Reshape dataset to be faster & more convenient
d=Data.to_numpy()
d=d.reshape((dataraw*datacol,))
d=d.reshape(int(d.shape[0]/8),8)
d=pd.DataFrame(d)
d=d.dropna()
d

,0,1,2,3,4,5,6,7
40,citrus fruit,tropical fruit,whole milk,butter,curd,yogurt,flour,bottled water
92,tropical fruit,root vegetables,other vegetables,frozen dessert,rolls/buns,flour,sweet spreads,salty snack
128,pork,berries,other vegetables,whole milk,whipped/sour cream,artif. sweetener,soda,abrasive cleaner
160,tropical fruit,root vegetables,whole milk,yogurt,domestic eggs,brown bread,pastry,sugar
172,butter milk,yogurt,cream cheese,spread cheese,rolls/buns,bottled water,soda,newspapers
...,...,...,...,...,...,...,...,...
39296,chicken,hamburger meat,citrus fruit,root vegetables,other vegetables,cream cheese,curd cheese,domestic eggs
39304,frankfurter,tropical fruit,other vegetables,whole milk,frozen meals,rolls/buns,detergent,napkins
39316,sausage,chicken,beef,hamburger meat,citrus fruit,grapes,root vegetables,whole milk
39317,butter,whipped/sour cream,flour,coffee,red/blush wine,salty snack,chocolate,hygiene articles


In [4]:
dsn=np.array(Data)
#This code is to convert database from shape(x,32) into shape to (x,2) to hold (transaction,item) 
arr=np.ndarray(shape=(dsn.shape[0]*dsn.shape[1], #new shape is number of rows*number of columns
                      2),dtype=object) #second dimention to hold transaction

#fill dataset to new shape (trans_id,item)
for i in range(0,len(dsn)):
    for j in range(len(dsn[i])):
        arr[(i*32)+j]=(i+1,dsn[i][j])
print("The new shape sample:")
arr

The new shape sample:


array([[1, 'tropical fruit'],
       [1, 'yogurt'],
       [1, 'coffee'],
       ...,
       [9834, nan],
       [9834, nan],
       [9834, nan]], dtype=object)

In [5]:
dsf=pd.DataFrame(arr,columns =['trans_id','item']).dropna()
dsf=dsf.groupby(['trans_id','item'])['item'].count().unstack().fillna(0)
dsf.to_csv("dsf.csv",index=False)

In [6]:

mapper={k: 1 + i for i, k in enumerate(dsf)}
mapper

{'Instant food products': 1,
 'UHT-milk': 2,
 'abrasive cleaner': 3,
 'artif. sweetener': 4,
 'baby cosmetics': 5,
 'baby food': 6,
 'bags': 7,
 'baking powder': 8,
 'bathroom cleaner': 9,
 'beef': 10,
 'berries': 11,
 'beverages': 12,
 'bottled beer': 13,
 'bottled water': 14,
 'brandy': 15,
 'brown bread': 16,
 'butter': 17,
 'butter milk': 18,
 'cake bar': 19,
 'candles': 20,
 'candy': 21,
 'canned beer': 22,
 'canned fish': 23,
 'canned fruit': 24,
 'canned vegetables': 25,
 'cat food': 26,
 'cereals': 27,
 'chewing gum': 28,
 'chicken': 29,
 'chocolate': 30,
 'chocolate marshmallow': 31,
 'citrus fruit': 32,
 'cleaner': 33,
 'cling film/bags': 34,
 'cocoa drinks': 35,
 'coffee': 36,
 'condensed milk': 37,
 'cooking chocolate': 38,
 'cookware': 39,
 'cream': 40,
 'cream cheese': 41,
 'curd': 42,
 'curd cheese': 43,
 'decalcifier': 44,
 'dental care': 45,
 'dessert': 46,
 'detergent': 47,
 'dish cleaner': 48,
 'dishes': 49,
 'dog food': 50,
 'domestic eggs': 51,
 'female sanitary pr

In [7]:
for row in range(len(d)):
    d.iloc[row]=d.iloc[row].map(mapper)
    
#len(d.iloc[0])
d

,0,1,2,3,4,5,6,7
40,32,159,167,17,42,168,55,14
92,159,125,104,60,124,55,154,130
128,113,11,104,167,163,4,140,3
160,159,125,167,168,51,16,107,153
172,18,168,41,152,124,14,140,97
...,...,...,...,...,...,...,...,...
39296,29,70,32,125,104,41,43,51
39304,58,159,104,167,63,124,47,96
39316,132,29,10,70,32,67,125,167
39317,17,163,55,36,121,130,30,75


In [8]:
X=d.to_numpy()

X.shape

(1834, 8)

In [9]:
def create_candidate_1(X):
    """
    create the 1-item candidate,
    it's basically creating a frozenset for each unique item
    and storing them in a list
    """
    c1 = []
    for transaction in X:
        for t in transaction:
            t = frozenset([t])
            if t not in c1:
                c1.append(t)
    return c1

In [10]:
def apriori(X, min_support):
    """
    pass in the transaction data and the minimum support 
    threshold to obtain the frequent itemset. Also
    store the support for each itemset, they will
    be used in the rule generation step
    """

    # the candidate sets for the 1-item is different,
    # create them independently from others
    c1 = create_candidate_1(X)
    freq_item, item_support_dict = create_freq_item(X, c1, min_support )
    freq_items = [freq_item]
    k = 0
    while len(freq_items[k]) > 0:
        freq_item = freq_items[k]
        ck = create_candidate_k(freq_item, k)       
        freq_item, item_support = create_freq_item(X, ck, min_support )
        freq_items.append(freq_item)
        item_support_dict.update(item_support)
        k += 1
        
    return freq_items, item_support_dict

In [11]:
def create_freq_item(X, ck, min_support):
    """
    filters the candidate with the specified
    minimum support
    """
    # loop through the transaction and compute
    # the count for each candidate (item)
    item_count = {}
    for transaction in X:
        for item in ck:
            if item.issubset(transaction):
                if item not in item_count:
                    item_count[item] = 1
                else: 
                    item_count[item] += 1    
    
    n_row = X.shape[0]
    freq_item = []
    item_support = {}
    
    # if the support of an item is greater than the 
    # min_support, then it is considered as frequent
    for item in item_count:
        support = item_count[item] / n_row
        if support >= min_support:
            freq_item.append(item)
        item_support[item] = support
        
    return freq_item, item_support

In [12]:
def create_candidate_k(freq_item, k):
    """create the list of k-item candidate"""
    ck = []
    
    # for generating candidate of size two (2-itemset)
    if k == 0:
        for f1, f2 in combinations(freq_item, 2):
            item = f1 | f2 # union of two sets
            ck.append(item)
    else:    
        for f1, f2 in combinations(freq_item, 2):       
            # if the two (k+1)-item sets has
            # k common elements then they will be
            # unioned to be the (k+2)-item candidate
            intersection = f1 & f2
            if len(intersection) == k:
                item = f1 | f2
                if item not in ck:
                    ck.append(item)
    return ck

In [13]:
#freq_items, item_support_dict = apriori(X, min_support = 0.005)
X=X.astype(int)
min_support = 0.1
freq_items, item_support_dict = apriori(X,min_support )

freq_items,

([[frozenset({32}),
   frozenset({159}),
   frozenset({167}),
   frozenset({17}),
   frozenset({42}),
   frozenset({168}),
   frozenset({14}),
   frozenset({125}),
   frozenset({104}),
   frozenset({124}),
   frozenset({113}),
   frozenset({163}),
   frozenset({140}),
   frozenset({51}),
   frozenset({16}),
   frozenset({107}),
   frozenset({90}),
   frozenset({111}),
   frozenset({58}),
   frozenset({65}),
   frozenset({132}),
   frozenset({29}),
   frozenset({10})],
  [frozenset({32, 167}),
   frozenset({159, 167}),
   frozenset({167, 168}),
   frozenset({104, 159}),
   frozenset({104, 125}),
   frozenset({104, 167}),
   frozenset({125, 167}),
   frozenset({124, 167}),
   frozenset({104, 168})],
  []],)

In [14]:
def create_rules(freq_items, item_support_dict, min_confidence):
    """
    create the association rules, the rules will be a list.
    each element is a tuple of size 4, containing rules'
    left hand side, right hand side, confidence and lift
    """
    association_rules = []

    # for the list that stores the frequent items, loop through
    # the second element to the one before the last to generate the rules
    # because the last one will be an empty list. It's the stopping criteria
    # for the frequent itemset generating process and the first one are all
    # single element frequent itemset, which can't perform the set
    # operation X -> Y - X
    for idx, freq_item in enumerate(freq_items[1:(len(freq_items) - 1)]):
        for freq_set in freq_item:
            
            # start with creating rules for single item on
            # the right hand side
            subsets = [frozenset([item]) for item in freq_set]
            rules, right_hand_side = compute_conf(freq_items, item_support_dict, 
                                                  freq_set, subsets, min_confidence)
            association_rules.extend(rules)
            
            # starting from 3-itemset, loop through each length item
            # to create the rules, as for the while loop condition,
            # e.g. suppose you start with a 3-itemset {2, 3, 5} then the 
            # while loop condition will stop when the right hand side's
            # item is of length 2, e.g. [ {2, 3}, {3, 5} ], since this
            # will be merged into 3 itemset, making the left hand side
            # null when computing the confidence
            if idx != 0:
                k = 0
                while len(right_hand_side[0]) < len(freq_set) - 1:
                    ck = create_candidate_k(right_hand_side, k = k)
                    rules, right_hand_side = compute_conf(freq_items, item_support_dict,
                                                          freq_set, ck, min_confidence)
                    association_rules.extend(rules)
                    k += 1    
    
    return association_rules

In [15]:
def compute_conf(freq_items, item_support_dict, freq_set, subsets, min_confidence):
    """
    create the rules and returns the rules info and the rules's
    right hand side (used for generating the next round of rules) 
    if it surpasses the minimum confidence threshold
    """
    rules = []
    right_hand_side = []
    
    for rhs in subsets:
        # create the left hand side of the rule
        # and add the rules if it's greater than
        # the confidence threshold
        lhs = freq_set - rhs
        conf = item_support_dict[freq_set] / item_support_dict[lhs]
        if conf >= min_confidence:
            lift = conf / item_support_dict[rhs]
            rules_info = lhs, rhs, conf, lift
            rules.append(rules_info)
            right_hand_side.append(rhs)
            
    return rules, right_hand_side

In [16]:
association_rules = create_rules(freq_items, item_support_dict, min_confidence = 0.04)
association_rules


[(frozenset({167}), frozenset({32}), 0.2059145673603505, 1.0179172952530533),
 (frozenset({32}), frozenset({167}), 0.5067385444743935, 1.0179172952530533),
 (frozenset({159}), frozenset({167}), 0.550755939524838, 1.1063377799436507),
 (frozenset({167}), frozenset({159}), 0.2792990142387733, 1.1063377799436505),
 (frozenset({167}), frozenset({168}), 0.35706462212486306, 1.0932496109799648),
 (frozenset({168}), frozenset({167}), 0.5442404006677796, 1.0932496109799648),
 (frozenset({159}), frozenset({104}), 0.5334773218142548, 1.1689335820876263),
 (frozenset({104}), frozenset({159}), 0.2951015531660693, 1.1689335820876263),
 (frozenset({125}), frozenset({104}), 0.5952380952380952, 1.3042612504978095),
 (frozenset({104}), frozenset({125}), 0.35842293906810035, 1.3042612504978095),
 (frozenset({167}), frozenset({104}), 0.5049288061336255, 1.10637924784835),
 (frozenset({104}), frozenset({167}), 0.5507765830346475, 1.10637924784835),
 (frozenset({167}), frozenset({125}), 0.3154435925520263,

In [17]:
n=np.array(association_rules)
inv_map = {v: k for k, v in mapper.items()}
inv_map

{1: 'Instant food products',
 2: 'UHT-milk',
 3: 'abrasive cleaner',
 4: 'artif. sweetener',
 5: 'baby cosmetics',
 6: 'baby food',
 7: 'bags',
 8: 'baking powder',
 9: 'bathroom cleaner',
 10: 'beef',
 11: 'berries',
 12: 'beverages',
 13: 'bottled beer',
 14: 'bottled water',
 15: 'brandy',
 16: 'brown bread',
 17: 'butter',
 18: 'butter milk',
 19: 'cake bar',
 20: 'candles',
 21: 'candy',
 22: 'canned beer',
 23: 'canned fish',
 24: 'canned fruit',
 25: 'canned vegetables',
 26: 'cat food',
 27: 'cereals',
 28: 'chewing gum',
 29: 'chicken',
 30: 'chocolate',
 31: 'chocolate marshmallow',
 32: 'citrus fruit',
 33: 'cleaner',
 34: 'cling film/bags',
 35: 'cocoa drinks',
 36: 'coffee',
 37: 'condensed milk',
 38: 'cooking chocolate',
 39: 'cookware',
 40: 'cream',
 41: 'cream cheese',
 42: 'curd',
 43: 'curd cheese',
 44: 'decalcifier',
 45: 'dental care',
 46: 'dessert',
 47: 'detergent',
 48: 'dish cleaner',
 49: 'dishes',
 50: 'dog food',
 51: 'domestic eggs',
 52: 'female sanitar

In [18]:
l=[]
for i in range(len(association_rules)):
    sets=association_rules[i][0:2]
    l.append([list(x) for x in sets])

for i in range(len(l)):
    for j in range(len(l[0])):
        l[i][j]=inv_map.get(l[i][j][0])
        n[i][j]=l[i][j]
final_results = pd.DataFrame(n)
final_results.columns={'Lift Side Item':0,'Right Side Item':1,'Conf':2,'Lift':3}
final_results

,Lift Side Item,Right Side Item,Conf,Lift
0,whole milk,citrus fruit,0.205915,1.017917
1,citrus fruit,whole milk,0.506739,1.017917
2,tropical fruit,whole milk,0.550756,1.106338
3,whole milk,tropical fruit,0.279299,1.106338
4,whole milk,yogurt,0.357065,1.09325
5,yogurt,whole milk,0.54424,1.09325
6,tropical fruit,other vegetables,0.533477,1.168934
7,other vegetables,tropical fruit,0.295102,1.168934
8,root vegetables,other vegetables,0.595238,1.304261
9,other vegetables,root vegetables,0.358423,1.304261
